## Clone ALBERT github

In [ ]:
!rm -rf albert
!git clone https://github.com/google-research/albert.git

Cloning into 'albert'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 347 (delta 0), reused 1 (delta 0), pack-reused 344
Receiving objects: 100% (347/347), 234.95 KiB | 3.09 MiB/s, done.
Resolving deltas: 100% (227/227), done.


## Installing necessary requirements

In [ ]:
!pip install -r albert/requirements.txt

In [ ]:
## Pulling data from Google drive 
from shutil import copyfile
copyfile('/content/drive/My Drive/w266_project/finBERT/finBert_pretrain_pr.tfrecord', 'finBert_pretrain_pr.tfrecord')
copyfile('/content/drive/My Drive/w266_project/finBERT/vocab.txt', 'vocab.txt')


'vocab.txt'

In [ ]:
!rm -rf 'albert_base_dir'
!mkdir 'albert_base_dir'

In [ ]:
copyfile('/content/drive/My Drive/w266_project/official_albert_base/albert_base/model.ckpt-best.data-00000-of-00001', 'albert_base_dir/model.ckpt.data-00000-of-00001')
copyfile('/content/drive/My Drive/w266_project/official_albert_base/albert_base/model.ckpt-best.index', 'albert_base_dir/model.ckpt.index')
copyfile('/content/drive/My Drive/w266_project/official_albert_base/albert_base/model.ckpt-best.meta', 'albert_base_dir/model.ckpt.meta')

'albert_base_dir/model.ckpt.meta'

In [ ]:
import os
import sys
sys.argv = sys.argv[:1]
import json
import nltk
import random
import logging
import tensorflow.compat.v1 as tf
import re
import sentencepiece as spm
import pprint


from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("albert")

from albert import modeling, optimization, tokenization
from albert.run_pretraining import input_fn_builder, model_fn_builder

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

#   # Upload credentials to TPU.
#   with open('/content/adc.json', 'r') as f:
#     auth_info = json.load(f)
#   tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

os.environ['COLAB_SKIP_TPU_AUTH'] = '1'

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False


TPU address is grpc://10.37.112.146:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 18249513612806564858),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3066059611994192841),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 18328675838490377172),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2011753602199702905),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10269956377614227717),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 16715912776761501291),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1126048719083901172),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8082265078761880496),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 28029666727

In [ ]:
# Testing out GS bucket
BUCKET_NAME = "finalbert" #@param {type:"string"}
MODEL_DIR = "albert_model" #@param {type:"string"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
PRETRAINDATA = "finBert_pretrain_pr.tfrecord"#@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)

if not BUCKET_NAME:
  log.warning("WARNING: BUCKET_NAME is not set. "
              "You will not be able to train the model.")
  
if BUCKET_NAME:
  !gsutil -m cp -r $PRETRAINDATA gs://$BUCKET_NAME/$PRETRAINING_DIR

Copying file://finBert_pretrain_pr.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/1.3 GiB.                                      


In [ ]:
# #padding our vocab list to make it 30000 to match checkpoint 

# # Build vocab
# def read_sentencepiece_vocab(filepath):
#   voc = []
#   with open('/content/drive/My Drive/w266_project/official_albert_base/albert_base/30k-clean.vocab', encoding='utf-8') as fi:
#     for line in fi:
#       voc.append(line.split("\t")[0])
#   # skip the first <unk> token
#   voc = voc[1:]
#   return voc

In [ ]:
if BUCKET_NAME:
  !gsutil -m cp -r "vocab.txt" $PRETRAINDATA gs://$BUCKET_NAME/$MODEL_DIR

Copying file://vocab.txt [Content-Type=text/plain]...
Copying file://finBert_pretrain_pr.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Resuming upload for file://finBert_pretrain_pr.tfrecord
Catching up md5 for file://finBert_pretrain_pr.tfrecord...
|
Operation completed over 2 objects/1.3 GiB.                       

In [ ]:
with open('vocab.txt','r') as f:
     n = sum(1 for line in f)
print (n)

30000


In [ ]:
# config file
#Vocab size should be same as vocab.txt size


finalbert_config = {
  "attention_probs_dropout_prob": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_hidden_groups": 1,
  "net_structure_type": 0,
  "gap_size": 0,
  "num_memory_blocks": 0,
  "inner_group_num": 1,
  "down_scale_factor": 1,
  "type_vocab_size": 2,
  "vocab_size": 30000
}

In [ ]:
with open("finalbert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(finalbert_config, fo, indent=2)
  

In [ ]:
if BUCKET_NAME:
  !gsutil -m cp -r "finalbert_config.json" $PRETRAINDATA gs://$BUCKET_NAME/$MODEL_DIR

Copying file://finalbert_config.json [Content-Type=application/json]...
Copying file://finBert_pretrain_pr.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 2 objects/1.3 GiB.                                      


In [ ]:
if BUCKET_NAME:
  !gsutil -m cp -r "vocab.txt" $PRETRAINDATA gs://$BUCKET_NAME/$MODEL_DIR

Copying file://vocab.txt [Content-Type=text/plain]...
Copying file://finBert_pretrain_pr.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 2 objects/1.3 GiB.                                      


In [ ]:
WEIGHT_CKPT = 'albert_model'
NEW_CKPT ='check_point'
BASE_CKPT = 'albert_base_dir'

if BUCKET_NAME:
  !gsutil -m cp  gs://$BUCKET_NAME/$BASE_CKPT/'model.ckpt.meta' gs://$BUCKET_NAME/$NEW_CKPT/'model.ckpt.meta'
  !gsutil -m cp  gs://$BUCKET_NAME/$BASE_CKPT/'model.ckpt.index' gs://$BUCKET_NAME/$NEW_CKPT/'model.ckpt.index'
  !gsutil -m cp  gs://$BUCKET_NAME/$BASE_CKPT/'model.ckpt.data-00000-of-00001' gs://$BUCKET_NAME/$NEW_CKPT/'model.ckpt.data-00000-of-00001'

Copying gs://finalbert/albert_base_dir/model.ckpt.meta [Content-Type=application/octet-stream]...
/ [1/1 files][791.0 KiB/791.0 KiB] 100% Done                                    
Operation completed over 1 objects/791.0 KiB.                                    
Copying gs://finalbert/albert_base_dir/model.ckpt.index [Content-Type=application/octet-stream]...
/ [1/1 files][  1.5 KiB/  1.5 KiB] 100% Done                                    
Operation completed over 1 objects/1.5 KiB.                                      
Copying gs://finalbert/albert_base_dir/model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
/ [1/1 files][ 45.1 MiB/ 45.1 MiB] 100% Done                                    
Operation completed over 1 objects/45.1 MiB.                                     


In [ ]:

BUCKET_NAME = "finalbert" #@param {type:"string"}
MODEL_DIR = "albert_model" #@param {type:"string"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
VOC_FNAME = "vocab.txt" #@param {type:"string"}

# Input data pipeline config
TRAIN_BATCH_SIZE =  32#@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 32#@param {type:"integer"}
LEARNING_RATE = 2e-05
TRAIN_STEPS =  1000000#@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "finalbert_config.json")

INIT_CHECKPOINT = 'gs://finalbert/check_point/model.ckpt'

bert_config = modeling.AlbertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))


INFO:tensorflow:Using checkpoint: gs://finalbert/check_point/model.ckpt
INFO:tensorflow:Using 2 data shards


In [ ]:
model_fn = model_fn_builder(
      albert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10000,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True,
      optimizer='lamb' ,
      poly_power= 0.5,
      start_warmup_step=1000)


from tensorflow.contrib import cluster_resolver as contrib_cluster_resolver
tpu_cluster_resolver =contrib_cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.estimator.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.estimator.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

INFO:tensorflow:Using config: {'_model_dir': 'gs://finalbert/albert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.37.112.146:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb49b08d7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.37.112.146:8470', '_evaluation_master': 'grpc://10.37.112.146:8470', '_is_chief': True, '_num_ps_replicas'

In [ ]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

INFO:tensorflow:Querying Tensorflow master (grpc://10.37.112.146:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 18249513612806564858)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 18328675838490377172)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2011753602199702905)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10269956377614227717)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 16715912776761501291)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:

In [ ]:
test_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=False)
estimator.evaluate(test_input_fn, steps = 20)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:<DatasetV1Adapter shapes: {input_ids: (4, 128), input_mask: (4, 128), masked_lm_ids: (4, 20), masked_lm_positions: (4, 20), masked_lm_weights: (4, 20), next_sentence_labels: (4, 1), segment_ids: (4, 128)}, types: {input_ids: tf.int32, input_mask: tf.int32, masked_lm_ids: tf.int32, masked_lm_positions: tf.int32, masked_lm_weights: tf.float32, next_sentence_labels: tf.int32, segment_ids: tf.int32}>
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:Found small feature: next_sentence_labels [4, 1]
INFO:tensorflow:*** Features *

{'global_step': 1000000,
 'loss': 3.242482,
 'masked_lm_accuracy': 0.51931065,
 'masked_lm_loss': 2.687353,
 'sentence_order_accuracy': 0.8234375,
 'sentence_order_loss': 0.41566944}

In [ ]:
# '''The max_seq_length in pre-train should be the same as create pre-train data step, 
#    for tensorflow TFRecord to parse in data correctly.'''
# fin_bert_dir = "gs://finalbert/"
# fin_bert_dir_model = 'albert_model'
# fin_bert_dir_base = 'albert_base_dir'

# !python -m albert.run_pretraining \
#     --input_file=input_files\
#     --output_dir=$fin_bert_dir/$fin_bert_dir_model/output \
#     --init_checkpoint="gs://finalbert/albert_base_dir/model.ckpt"\
#     --albert_config_file="gs://finalbert/albert_model/finalbert_config.json"\
#     --do_train \
#     --do_eval \
#     --train_batch_size=512 \
#     --eval_batch_size=64 \
#     --max_seq_length=128 \
#     --max_predictions_per_seq=20 \
#     --optimizer='lamb' \
#     --learning_rate=.00176 \
#     --num_train_steps=10000 \
#     --num_warmup_steps=3125 \
#     --save_checkpoints_steps=5000 \
#     --use_tpu=True \
#     --tpu_name=TPU_ADDRESS \

